## Optimizing LSTM using Genetic Algorithm 

#### Importing required packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

from keras.layers import LSTM, Input, Dense
from keras.models import Sequential
from keras.layers import *
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray
import operator
import random


np.random.seed(1120)
import warnings
#warnings.filterwarnings('ignore')

Using TensorFlow backend.
C:\Users\Ali\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


### France

In [2]:
# France
df = pd.read_excel('LSTM.xlsx', usecols=[1], nrows=5000)
#df=df[0:50000]
#df=df[0:2500]
df.shape

(5000, 1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 1 columns):
Consumption    5000 non-null int64
dtypes: int64(1)
memory usage: 39.1 KB


### Scaling

In [4]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(df)
df=pd.DataFrame(dataset)
df.columns=['Consumption']
df.head()

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Consumption
0,0.500748
1,0.507280
2,0.507070
3,0.504814
4,0.454393


## Defining Functions

In [5]:
def train_evaluate(ga_individual_solution):  
    df1= df.copy(deep=True)
    print('Dataset Shape',df1.shape)

    # Decode GA solution to integer for window_size and num_units
    
    # Parameter 1
    no_wins = ga_individual_solution[0:2]
    no_wins = int(''.join(str(x) for x in no_wins), base=2)
    
    if no_wins==0:no_wins+=1
    elif no_wins==1:no_wins+=1  
    elif no_wins==2:no_wins+=1
    elif no_wins==3:no_wins+=1 
    
    print('Number of Windows are:\n',no_wins)
    
    # Parameter 2
    # Windows starting points
    s1=BitArray(ga_individual_solution[2:9]).uint #7 bits
    s2=BitArray(ga_individual_solution[9:18]).uint # 9 bits
    s3=BitArray(ga_individual_solution[18:28]).uint # 10 bits
    s4=BitArray(ga_individual_solution[28:39]).uint # 11 bits
    
    #  Windows length, max=31
    l1= BitArray(ga_individual_solution[39:44]).uint
    l2= BitArray(ga_individual_solution[44:49]).uint
    l3= BitArray(ga_individual_solution[49:54]).uint
    l4= BitArray(ga_individual_solution[54:59]).uint
    
    # Batch Size
    batch_size = BitArray(ga_individual_solution[59:66]).uint
    batch_size+=10
    print('Batch Size is:',batch_size)
    
    act = ga_individual_solution[66:68]
    act = int(''.join(str(x) for x in act), base=2)
    
    opt = ga_individual_solution[68:70]
    opt = int(''.join(str(x) for x in opt), base=2)
    
    # Activation Decode
    if act==0:act='tanh'
    elif act==1:act='relu'
    elif act==2:act='selu'
    else:act='elu'         
        
    #  Optimizer Decode
    if opt==0:opt='sgd'
    elif opt==1:opt='RMSprop'
    elif opt==2:opt='Adamax'
    else:opt='Adam'     
        
    length=[l1,l2,l3,l4]
    new_list = [n+1 if n==0 else n for n in length]
    l1,l2,l3,l4=new_list[0:4]
    #window_length=max(new_list)

    # Avoid Overlapping 1 & 2
    
    if s2-(s1+l1) <=-10:
        print('Overlap 1 and 2')
        return 0.80,

        #1 & 3
    elif s3-(s1+l1) <=-10:
        print('Overlap 1 and 3')
        return 0.80,

        # 1 & 4
    elif s4-(s1+l1)<=-10:
        print('Overlap 1 and 4')
        return 0.80,

    elif s3-(s2+l2) <=-10:
        print('Overlap 2 and 3')
        return 0.80,
              
    elif s4-(s2+l2) <=-10:
        print('Overlap 2 and 4')
        return 0.80,

    elif s4-(s3+l3) <=-10:
        print('Overlap 3 and 4')
        return 0.80,

    else:
        print('No Overlap')
    
    print('Windows length are:\n',l1,l2,l3,l4)
        
    print('1st Window Start:',s1)
    print('2nd Window Start:',s2)
    print('3rd Window Start:',s3)
    print('4th Window Start:',s4) 
    
    print('Activation Function is:',act)
    print('Optimizer is:',opt)
     
    # Return fitness score of 100 if window_size or num_unit is zero
    if batch_size==0:
        return 1, 
    
    if no_wins==1:
        lag=np.arange(s1,s1+l1,1)
        window_length=l1
        print('Only Lag is\n',lag)
        
    elif no_wins==2:
        lag1=np.arange(s1,s1+l1,1).tolist()
        lag2=np.arange(s2,s2+l2,1).tolist()
        lag=[lag1,lag2]
        lag = pad_sequences(lag, padding='post',value=1)
        lag = [item for sublist in lag for item in sublist]
        window_length=max(l1,l2)
        print('Two Lags are\n',lag)

    elif no_wins==3:
        lag1=np.arange(s1,s1+l1,1).tolist()
        lag2=np.arange(s2,s2+l2,1).tolist()
        lag3=np.arange(s3,s3+l3,1).tolist()
        lag=[lag1,lag2,lag3]
        lag = pad_sequences(lag, padding='post',value=1)
        lag = [item for sublist in lag for item in sublist]
        print('Three Lags are\n',lag)
        window_length=max(l1,l2,l3)

    elif no_wins==4:
        lag1=np.arange(s1,s1+l1,1).tolist()
        lag2=np.arange(s2,s2+l2,1).tolist()
        lag3=np.arange(s3,s3+l3,1).tolist()
        lag4=np.arange(s4,s4+l4,1).tolist()
        lag=[lag1,lag2,lag3,lag4]
        lag = pad_sequences(lag, padding='post',value=1)
        lag = [item for sublist in lag for item in sublist]
        window_length=max(l1,l2,l3,l4)
        print('Four Lags are\n',lag)
      
    print('Lags length:\n', len(lag))
    print('Length Used:\n', window_length)
    for col in df1.columns:
        for idx, l in enumerate(lag):
            df1.loc[:,col+"_"+str(idx)] = df1[col].shift(l)
           
    df1.dropna(how='any',inplace=True)
    dataset = df1.values # Converted dataframe to numpy ndarray
    dataset = dataset.astype('float32')
    print('New Dataset shape after Lags:',dataset.shape)
    
    # split into train and test sets
    train_size = int(len(dataset) * 0.80)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    print('Training and test data length',len(train), len(test))
    
    trainX=train[:,1:]
    trainY=train[:,0]
    testX=test[:,1:]
    testY=test[:,0]
    
    print('trainX.shape[0]: ',trainX.shape[0])
    print('window_length: ',window_length) 
    print('no_wins: ',no_wins)
    
    print('trainX shape:\n',trainX.shape)
    
    #LSTM
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0],window_length,(no_wins)))
    testX = np.reshape(testX, (testX.shape[0],window_length,(no_wins)))
    print('Train X shape after np.reshape',trainX.shape)
    print('Test X shape after np.reshape',testX.shape)
    print('Train Y Shape',trainY.shape)
    print('Test Y Shape',testY.shape)
      
    # Train LSTM model and predict on validation set
    #inputs = Input(shape=(window_size,1))
    model = Sequential() # New Instance of Model Object
    model.add(LSTM(10, input_shape=(window_length,(no_wins)))) 
    #model.add(Dense(10, activation=act))
    #model.add(Dense(60, activation=act))
    model.add(Dense(10, activation=act))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=opt,loss='mean_squared_error')
    #model.fit(trainX, trainY, epochs=1, batch_size=10,shuffle=True)
    
    start = time.time()
    model.fit(trainX, trainY, epochs=1, shuffle=False,batch_size=batch_size,
              validation_data=(testX, testY), 
              callbacks=[EarlyStopping(monitor='val_loss', patience=20)], verbose=1)
    
    y_pred = model.predict(testX)
    end = time.time()
    
    # Calculate the RMSE score as fitness score for GA
    rmse = np.sqrt(mean_squared_error(testY, y_pred))
    print('Validation RMSE: ', rmse,'\n')
    print ("Model took %0.2f seconds to train"%(end - start))
    return rmse,

In [6]:
# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0

creator.create('FitnessMin', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMin)

In [7]:
# self
#population_size = 20
#num_generations = 30

population_size = 10
num_generations = 30

#gene_length = 69
gene_length = 70


toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1) #Generates single RAND
toolbox.register('individual', tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, n=gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

In [8]:
print(len(toolbox.individual()))
print(len(toolbox.population(population_size)))

70
10


In [9]:
'''population_size = 2
num_generations = 3
gene_length = 24


toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5) #Generates single bernoulli
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n=gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)'''

"population_size = 2\nnum_generations = 3\ngene_length = 24\n\n\ntoolbox = base.Toolbox()\ntoolbox.register('binary', bernoulli.rvs, 0.5) #Generates single bernoulli\ntoolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n=gene_length)\ntoolbox.register('population', tools.initRepeat, list , toolbox.individual)"

## Algorithm

*deap.algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen[, stats, halloffame, verbose])*

In [10]:
# Register the crossover operator
toolbox.register('mate', tools.cxUniform,indpb= 0.6)

# Register a mutation operator
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.1)

# Operator for selecting individuals for breeding
#toolbox.register('select', tools.selRoulette)
toolbox.register('select',tools.selTournament,tournsize=4)

# Register the evaluation function
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)

In [11]:
from keras.callbacks import EarlyStopping
import time

In [12]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.1, ngen = num_generations,
                                       stats=stats, halloffame=hof, verbose=True)

Dataset Shape (5000, 1)
Number of Windows are:
 2
Batch Size is: 121
Overlap 3 and 4
Dataset Shape (5000, 1)
Number of Windows are:
 3
Batch Size is: 103
Overlap 3 and 4
Dataset Shape (5000, 1)
Number of Windows are:
 1
Batch Size is: 98
Overlap 1 and 4
Dataset Shape (5000, 1)
Number of Windows are:
 2
Batch Size is: 81
Overlap 1 and 2
Dataset Shape (5000, 1)
Number of Windows are:
 2
Batch Size is: 26
Overlap 3 and 4
Dataset Shape (5000, 1)
Number of Windows are:
 4
Batch Size is: 21
No Overlap
Windows length are:
 28 26 2 27
1st Window Start: 36
2nd Window Start: 72
3rd Window Start: 799
4th Window Start: 966
Activation Function is: relu
Optimizer is: sgd
Four Lags are
 [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 1, 1, 799, 800, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 966, 967

Train on 3592 samples, validate on 899 samples
Epoch 1/1
3592/3592 [==============================] - 2s 506us/step - loss: 0.2724 - val_loss: 0.1004
Validation RMSE:  0.3168504 

Model took 2.69 seconds to train
Dataset Shape (5000, 1)
Number of Windows are:
 2
Batch Size is: 62
No Overlap
Windows length are:
 27 17 1 12
1st Window Start: 92
2nd Window Start: 493
3rd Window Start: 567
4th Window Start: 976
Activation Function is: relu
Optimizer is: Adamax
Two Lags are
 [92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Lags length:
 54
Length Used:
 27
New Dataset shape after Lags: (4491, 55)
Training and test data length 3592 899
trainX.shape[0]:  3592
window_length:  27
no_wins:  2
trainX shape:
 (3592, 54)
Train X shape after np.reshape (3592, 27, 2)
Test X shape after np.reshape (899, 27, 2)


Train on 3616 samples, validate on 905 samples
Epoch 1/1
3616/3616 [==============================] - 2s 508us/step - loss: 0.0886 - val_loss: 0.0293
Validation RMSE:  0.17113975 

Model took 2.67 seconds to train
Dataset Shape (5000, 1)
Number of Windows are:
 2
Batch Size is: 30
No Overlap
Windows length are:
 27 19 1 8
1st Window Start: 20
2nd Window Start: 365
3rd Window Start: 535
4th Window Start: 960
Activation Function is: relu
Optimizer is: sgd
Two Lags are
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 1, 1, 1, 1, 1, 1, 1, 1]
Lags length:
 54
Length Used:
 27
New Dataset shape after Lags: (4617, 55)
Training and test data length 3693 924
trainX.shape[0]:  3693
window_length:  27
no_wins:  2
trainX shape:
 (3693, 54)
Train X shape after np.reshape (3693, 27, 2)
Test X shape after np.reshape (924, 27, 2)
Train Y Shape (369

Train on 2594 samples, validate on 649 samples
Epoch 1/1
2594/2594 [==============================] - 2s 963us/step - loss: 0.0410 - val_loss: 0.0213
Validation RMSE:  0.14581662 

Model took 3.51 seconds to train
Dataset Shape (5000, 1)
Number of Windows are:
 2
Batch Size is: 62
No Overlap
Windows length are:
 27 17 1 13
1st Window Start: 84
2nd Window Start: 460
3rd Window Start: 566
4th Window Start: 972
Activation Function is: elu
Optimizer is: Adam
Two Lags are
 [84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Lags length:
 54
Length Used:
 27
New Dataset shape after Lags: (4524, 55)
Training and test data length 3619 905
trainX.shape[0]:  3619
window_length:  27
no_wins:  2
trainX shape:
 (3619, 54)
Train X shape after np.reshape (3619, 27, 2)
Test X shape after np.reshape (905, 27, 2)
Train Y Sh

Train on 3628 samples, validate on 907 samples
Epoch 1/1
3628/3628 [==============================] - 3s 739us/step - loss: 0.0647 - val_loss: 0.0121
Validation RMSE:  0.110192806 

Model took 3.88 seconds to train
Dataset Shape (5000, 1)
Number of Windows are:
 3
Batch Size is: 72
No Overlap
Windows length are:
 14 1 1 12
1st Window Start: 77
2nd Window Start: 168
3rd Window Start: 540
4th Window Start: 1848
Activation Function is: elu
Optimizer is: Adamax
Three Lags are
 [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 168, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 540, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Lags length:
 42
Length Used:
 14
New Dataset shape after Lags: (4460, 43)
Training and test data length 3568 892
trainX.shape[0]:  3568
window_length:  14
no_wins:  3
trainX shape:
 (3568, 42)
Train X shape after np.reshape (3568, 14, 3)
Test X shape after np.reshape (892, 14, 3)
Train Y Shape (3568,)
Test Y Shape (892,)
Train on 3568 samples, validate on 892 samples
Epoch 

...

Train on 3565 samples, validate on 892 samples
Epoch 1/1
3565/3565 [==============================] - 15s 4ms/step - loss: 0.0314 - val_loss: 0.0083
Validation RMSE:  0.09132967 

Model took 21.45 seconds to train
Dataset Shape (5000, 1)
Number of Windows are:
 3
Batch Size is: 72
No Overlap
Windows length are:
 30 1 1 12
1st Window Start: 77
2nd Window Start: 136
3rd Window Start: 669
4th Window Start: 1840
Activation Function is: elu
Optimizer is: Adamax
Three Lags are
 [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 136, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 669, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Lags length:
 90
Length Used:
 30
New Dataset shape after Lags: (4331, 91)
Training and test data length 3464 867
trainX.shape[0]:  3464
window_length:  30
no_wins:  3
trainX shape:
 (3464, 90)
Train X shape aft

New Dataset shape after Lags: (4075, 91)
Training and test data length 3260 815
trainX.shape[0]:  3260
window_length:  30
no_wins:  3
trainX shape:
 (3260, 90)
Train X shape after np.reshape (3260, 30, 3)
Test X shape after np.reshape (815, 30, 3)
Train Y Shape (3260,)
Test Y Shape (815,)
Train on 3260 samples, validate on 815 samples
Epoch 1/1
3260/3260 [==============================] - 14s 4ms/step - loss: 0.1396 - val_loss: 0.0174
Validation RMSE:  0.13207799 

Model took 20.27 seconds to train
Dataset Shape (5000, 1)
Number of Windows are:
 3
Batch Size is: 72
No Overlap
Windows length are:
 30 1 1 12
1st Window Start: 77
2nd Window Start: 136
3rd Window Start: 669
4th Window Start: 1840
Activation Function is: elu
Optimizer is: Adamax
Three Lags are
 [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 136, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 669, 1, 1, 1,

3464/3464 [==============================] - 16s 5ms/step - loss: 0.0691 - val_loss: 0.0083
Validation RMSE:  0.091084704 

Model took 22.13 seconds to train
Dataset Shape (5000, 1)
Number of Windows are:
 3
Batch Size is: 72
No Overlap
Windows length are:
 30 1 1 12
1st Window Start: 77
2nd Window Start: 136
3rd Window Start: 669
4th Window Start: 1840
Activation Function is: elu
Optimizer is: Adamax
Three Lags are
 [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 136, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 669, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Lags length:
 90
Length Used:
 30
New Dataset shape after Lags: (4331, 91)
Training and test data length 3464 867
trainX.shape[0]:  3464
window_length:  30
no_wins:  3
trainX shape:
 (3464, 90)
Train X shape after np.reshape (3464, 30, 3)
Test X shape after np.reshap

New Dataset shape after Lags: (4331, 91)
Training and test data length 3464 867
trainX.shape[0]:  3464
window_length:  30
no_wins:  3
trainX shape:
 (3464, 90)
Train X shape after np.reshape (3464, 30, 3)
Test X shape after np.reshape (867, 30, 3)
Train Y Shape (3464,)
Test Y Shape (867,)
Train on 3464 samples, validate on 867 samples
Epoch 1/1
3464/3464 [==============================] - 16s 5ms/step - loss: 0.0686 - val_loss: 0.0105
Validation RMSE:  0.1023478 

Model took 22.32 seconds to train
Dataset Shape (5000, 1)
Number of Windows are:
 3
Batch Size is: 72
No Overlap
Windows length are:
 30 1 1 12
1st Window Start: 77
2nd Window Start: 136
3rd Window Start: 669
4th Window Start: 1840
Activation Function is: elu
Optimizer is: Adamax
Three Lags are
 [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 136, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 669, 1, 1, 1, 

New Dataset shape after Lags: (4331, 91)
Training and test data length 3464 867
trainX.shape[0]:  3464
window_length:  30
no_wins:  3
trainX shape:
 (3464, 90)
Train X shape after np.reshape (3464, 30, 3)
Test X shape after np.reshape (867, 30, 3)
Train Y Shape (3464,)
Test Y Shape (867,)
Train on 3464 samples, validate on 867 samples
Epoch 1/1
3464/3464 [==============================] - 16s 5ms/step - loss: 0.0187 - val_loss: 0.0029
Validation RMSE:  0.054292 

Model took 23.29 seconds to train
Dataset Shape (5000, 1)
Number of Windows are:
 3
Batch Size is: 72
No Overlap
Windows length are:
 30 1 1 12
1st Window Start: 77
2nd Window Start: 136
3rd Window Start: 669
4th Window Start: 1840
Activation Function is: elu
Optimizer is: Adamax
Three Lags are
 [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 136, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 669, 1, 1, 1, 1

In [ ]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.1, ngen = num_generations,
                                       stats=stats, halloffame=hof, verbose=True)

### Print top N solutions

In [ ]:
a=tools.selBest(population,k = 1)
a=np.asarray(a)
a

### Decoder

In [ ]:
best_individuals = tools.selBest(population,k = 30)
#best_window_size = None
#best_num_units = None

for bi in best_individuals:  
    
    nu_wins = bi[0:2]
    nu_wins = int(''.join(str(x) for x in nu_wins), base=2)
    if nu_wins==0:
        nu_wins+=1
    elif nu_wins==1:
        nu_wins+=1  
    elif nu_wins==2:
        nu_wins+=1
    elif nu_wins==3:
        nu_wins+=1 
    
    
    
    win_len_bits= BitArray(bi[2:6])
    first_win_bits=BitArray(bi[6:17])
    sec_win_bits=BitArray(bi[17:28])
    third_win_bits=BitArray(bi[28:39])
    fourth_win_bits=BitArray(bi[39:50])
      
    
    num_units_bits = BitArray(bi[50:56]) 
    BatchSize_bits = BitArray(bi[56:63]) 
    
    
    act = bi[63:65]
    act = int(''.join(str(x) for x in act), base=2)
    
    opt = bi[65:67]
    opt = int(''.join(str(x) for x in opt), base=2)
    
    # Activation Decode
    if act==0:
        act='tanh'
    elif act==1:
        act='relu'
    elif act==2:
        act='selu'
    else:
        act='elu'         

        
    #  Optimizer Decode
    if opt==0:
        opt='sgd'
    elif opt==1:
        opt='RMSprop'
    elif opt==2:
        opt='Adamax'
    else:
        opt='Adam'  
           
    window_length = win_len_bits.uint
    first_win=first_win_bits.uint
    sec_win=sec_win_bits.uint
    third_win=third_win_bits.uint
    fourth_win=fourth_win_bits.uint  
    
    
    if window_length==0:
        window_length+=1
    
    if first_win-sec_win in list(range(0,21)):
        sec_win+=40
    if first_win-sec_win in list(range(-21,0)):
        sec_win+=20
    
    if first_win-third_win in list(range(0,21)):
        third_win+=40
    if first_win-third_win in list(range(-21,0)):
        third_win+=20  
    
    if first_win-fourth_win in list(range(0,21)):
        fourth_win+=40
    if first_win-fourth_win in list(range(-21,0)):
        fourth_win+=20
        
    if sec_win-third_win in list(range(0,21)):
        third_win+=40
    if sec_win-third_win in list(range(-21,0)):
        third_win+=20
        
    if sec_win-fourth_win in list(range(0,21)):
        fourth_win+=40
    if sec_win-fourth_win in list(range(-21,0)):
        fourth_win+=20
        
    if third_win-fourth_win in list(range(0,21)):
        fourth_win+=40
    if third_win-fourth_win in list(range(-21,0)):
        fourth_win+=20
    
    best_no_of_windows = nu_wins
    best_window_length=window_length
    
    num_units = num_units_bits.uint
    if num_units==0:
        num_units+=1
    best_num_units=num_units
    
    batchsize=BatchSize_bits.uint
    if batchsize==0:
        batchsize+=1
    best_batchsize=batchsize
    
    best_activation=act
    best_optimizer=opt
    
    
    print('\n*No of Windows: ', nu_wins, 'Window Length:',best_window_length,
          'First Window Start: ',first_win,'Second Window Start: ',sec_win,
          'Third Window Start: ',third_win, 'Fourth Window Start: ',fourth_win,
          'Num of LSTM Units: ', best_num_units, 'Batch Size: ',best_batchsize, 
          'Activation: ',best_activation, "Optimizer: ",best_optimizer)

## Modified

In [ ]:
# Modified
best_individuals = tools.selBest(population,k = 20)
#best_window_size = None
#best_num_units = None

for bi in best_individuals:  
    
    nu_wins = bi[0:2]
    nu_wins = int(''.join(str(x) for x in nu_wins), base=2)
    if nu_wins==0:
        nu_wins+=1
    elif nu_wins==1:
        nu_wins+=1  
    elif nu_wins==2:
        nu_wins+=1
    elif nu_wins==3:
        nu_wins+=1 
    
    
    
    win_len_bits= BitArray(bi[2:6])
    first_win_bits=BitArray(bi[6:16])
    sec_win_bits=BitArray(bi[16:26])
    third_win_bits=BitArray(bi[26:36])
    fourth_win_bits=BitArray(bi[36:46])
      
    
    num_units_bits = BitArray(bi[46:52]) 
    BatchSize_bits = BitArray(bi[52:59]) 
    
    
    act = bi[59:61]
    act = int(''.join(str(x) for x in act), base=2)
    
    opt = bi[61:63]
    opt = int(''.join(str(x) for x in opt), base=2)
    
    # Activation Decode
    if act==0:
        act='tanh'
    elif act==1:
        act='relu'
    elif act==2:
        act='selu'
    else:
        act='elu'         

        
    #  Optimizer Decode
    if opt==0:
        opt='sgd'
    elif opt==1:
        opt='RMSprop'
    elif opt==2:
        opt='Adamax'
    else:
        opt='Adam'  
           
    window_length = win_len_bits.uint
    first_win=first_win_bits.uint
    sec_win=sec_win_bits.uint
    third_win=third_win_bits.uint
    fourth_win=fourth_win_bits.uint  
    
    if window_length==0:
        window_length+=1
    
    #1 & 2
    if first_win-sec_win in list(range(0,41)):
        if sec_win>48:
            sec_win-=48
        else:
            sec_win+=96
    if first_win-sec_win in list(range(-39,0)):
        sec_win+=30
    
    #1 & 3
    if first_win-third_win in list(range(0,41)):
        if third_win>48:
            third_win-=48
        else:
            third_win+=96
    if first_win-third_win in list(range(-39,0)):
        third_win+=30  
        
    # 1 & 4
    if first_win-fourth_win in list(range(0,41)):
        if fourth_win>48:
            fourth_win-=48
        else:
            fourth_win+=96
            
    if first_win-fourth_win in list(range(-39,0)):
        fourth_win+=30
    
    #2 & 3
    if sec_win-third_win in list(range(0,41)):
        if third_win>48:
            third_win-=48
        else:
            third_win+=96
    if sec_win-third_win in list(range(-39,0)):
        third_win+=30
    #Rep
    if first_win-third_win in list(range(-16,16)):
        print('error') 
              
    #
     #2 & 4   
    if sec_win-fourth_win in list(range(0,41)):
        if fourth_win>48:
            fourth_win-=48
        else:
            fourth_win+=96
    if sec_win-fourth_win in list(range(-39,0)):
        fourth_win+=30
    #Rep
    if first_win-fourth_win in list(range(-16,16)):
        print('error') 
        
    
    # 3 & 4    
    if third_win-fourth_win in list(range(0,41)):
        if fourth_win>48:
            fourth_win-=48
        else:
            fourth_win+=96
    if third_win-fourth_win in list(range(-39,0)):
        fourth_win+=30
    #Rep
    if first_win-fourth_win in list(range(-16,16)):
        print('error') 
    if sec_win-fourth_win in list(range(-16,16)):
        print('error') 
    
    
    best_no_of_windows = nu_wins
    best_window_length=window_length
    
    
    
    num_units = num_units_bits.uint
    num_units+=10
    if num_units<=5:
        num_units+=10
    best_num_units=num_units

    
    batch_size=BatchSize_bits.uint
    batch_size+=20
    best_batchsize=batch_size
    
    best_activation=act
    best_optimizer=opt
    
    
    print('\n*No of Windows: ', nu_wins, 'Window Length:',best_window_length,
          'First Window Start: ',first_win,'Second Window Start: ',sec_win,
          'Third Window Start: ',third_win, 'Fourth Window Start: ',fourth_win,
          'Num of LSTM Units: ', best_num_units, 'Batch Size: ',best_batchsize, 
          'Activation: ',best_activation, "Optimizer: ",best_optimizer)

In [ ]:
gen1,avg1,min1, max1 = log.select('gen','avg','min','max')

## Plots

In [ ]:
sns.set_style('white')
sns.set_context("paper", font_scale=1.2)  

plt.figure(figsize=(8,4))
#avg1,min1,max1=scaler.inverse_transform([avg1,min1,max1])

plt.plot(gen1,avg1,marker='s',linewidth=1,markersize=6,label='Average')
plt.plot(gen1,min1,marker='d',linewidth=1,markersize=6,label='Minimum')
#plt.plot(gen1,max1,marker='*',linewidth=1,markersize=8,label='Maximum')
#plt.tick_params(left=False, labelleft=True)
#plt.box(False)
plt.margins(0.2)


plt.xlabel('Generation Number',labelpad=15)
plt.ylabel('RMSE LSTM')
#plt.xlim(-1, 26)
plt.legend(loc='best');
sns.despine(left=True)
plt.tight_layout()
plt.savefig('Fig1.png', bbox_inches='tight', dpi=400) 
plt.show()

In [ ]:
# Box Plot
a,b,c=log.select('min','max','std')

#a,b,c=scaler.inverse_transform([a,b,c])

df=pd.DataFrame({'Minimum':a,'Maximum':b, 'Std':c})

plt.figure(figsize=(6,4))
sns.set_style('white')
sns.set_context("paper", font_scale=1.3)  
sns.boxplot(data=df, width=0.6,color='#1f77b4')
plt.xlabel('Statistics across Generations and Particles',labelpad=15)
plt.ylabel('RMSE LSTM')
sns.despine(left=True)
plt.tight_layout()
plt.savefig('Fig1.png', bbox_inches='tight', dpi=400) 
plt.show()

### Save Results

In [ ]:
a,b,c,d,e=log.select('gen','avg','min','max','std')

#b,c,d,e=scaler.inverse_transform([b,c,d,e])
df=pd.DataFrame({'Generation':a,'Average':b, 'Minimum':c,'Maximum':d,'Std':e})
writer = pd.ExcelWriter('GA_Final__norm.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

### Functional Check Only

In [ ]:
# Decode GA solution to integer for window_size and num_units
    
#no_wins_bit = BitArray(ga_individual_solution[0:2])
no_wins = ga_individual_solution[0:2]
no_wins = int(''.join(str(x) for x in no_wins), base=2)
if no_wins==0:
    no_wins+=1
elif no_wins==1:
    no_wins+=1  
elif no_wins==2:
    no_wins+=1
elif no_wins==3:
    no_wins+=1  
print('Number of windows are:',no_wins)
    
win_len_bits= BitArray(ga_individual_solution[2:6])
first_win_bits=BitArray(ga_individual_solution[6:17])
sec_win_bits=BitArray(ga_individual_solution[17:28])
third_win_bits=BitArray(ga_individual_solution[28:39])
fourth_win_bits=BitArray(ga_individual_solution[39:50])
    
num_units_bits = BitArray(ga_individual_solution[50:56]) 
BatchSize_bits = BitArray(ga_individual_solution[56:63]) 
    
act = ga_individual_solution[63:65]
act = int(''.join(str(x) for x in act), base=2)
    
opt = ga_individual_solution[65:67]
opt = int(''.join(str(x) for x in opt), base=2)
    
# Activation Decode
if act==0:
    act='tanh'
elif act==1:
    act='relu'
elif act==2:
    act='selu'
else:
    act='elu'         

        
    #  Optimizer Decode
if opt==0:
    opt='sgd'
elif opt==1:
    opt='RMSprop'
elif opt==2:
    opt='Adamax'
else:
    opt='Adam'
        
#no_wins= no_wins_bit.uint 
window_length = win_len_bits.uint
first_win =first_win_bits.uint

sec_win= sec_win_bits.uint

third_win =first_win_bits.uint


fourth_win =fourth_win_bits.uint  

if first_win==sec_win:
    sec_win+=window_length
if first_win==third_win:
    third_win+=window_length
if first_win==fourth_win:
    fourth_win+=window_length
if sec_win==third_win:
    third_win+=window_length
if sec_win==fourth_win:
    fourth_win+=window_length
if third_win==fourth_win:
    fourth_win+=window_length
    
print('1st',first_win)
print('2nd',sec_win)
print('3rd',third_win)
print('4th',fourth_win)


num_units = num_units_bits.uint
batch_size=BatchSize_bits.uint
    
    
print('\nNo of Windows: ', no_wins+1, ',Window Length: ',
          window_length, 'First Win start: ',first_win, 'Second Win start: ',sec_win,
          'Third Win start: ', third_win, 'Fourth Win start: ',fourth_win,
          'No of LSTM units: ',num_units, 'Batch Size: ',batch_size, 
          'Activation Function:',act, 'Optimizer:',opt)
    
    # Return fitness score of 100 if window_size or num_unit is zero
#if window_length == 0 or num_units == 0:
#return 100, 
    
if no_wins==1:
    lag=np.arange(first_win,first_win+window_length,1)
    
elif no_wins==2:
    lag1=np.arange(first_win,first_win+window_length,1)
    lag2=np.arange(sec_win,sec_win+window_length,1)
    lag=np.concatenate([lag1,lag2])
elif no_wins==3:
    lag1=np.arange(first_win,first_win+window_length,1)
    lag2=np.arange(sec_win,sec_win+window_length,1)
    lag3=np.arange(third_win,third_win+window_length,1)
    lag=np.concatenate([lag1,lag2,lag3])

elif no_wins==4:
    lag1=np.arange(first_win,first_win+window_length,1)
    lag2=np.arange(sec_win,sec_win+window_length,1)
    lag3=np.arange(third_win,third_win+window_length,1)
    lag4=np.arange(fourth_win,fourth_win+window_length,1)
    lag=np.concatenate([lag1,lag2,lag3,lag4])


print('Dataset shape original',df.shape)
print('Lag is', lag)
for col in df.columns:
    for l in lag:
        df.loc[:,col+"_"+str(l)] = df[col].shift(l)
        
df.dropna(how='any',inplace=True)
dataset = df.values # Converted dataframe to numpy ndarray
dataset = dataset.astype('float32')
print('Dataset shape after',dataset.shape)
    
# split into train and test sets
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))


trainX=train[:,1:]
trainY=train[:,0]
testX=test[:,1:]
testY=test[:,0]
 

    
#LSTM
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0],window_length,(no_wins)))
testX = np.reshape(testX, (testX.shape[0],window_length,(no_wins)))
print('Train X shape after np.reshape',trainX.shape)
print('Test X shape after np.reshape',testX.shape)
print('Train Y Shape',trainY.shape)
print('Test Y Shape',testY.shape)
  
   
    
# Train LSTM model and predict on validation set
#inputs = Input(shape=(window_size,1))
model = Sequential() # New Instance of Model Object
model.add(LSTM(num_units, input_shape=(window_length,(no_wins))))
model.add(Dense(30, activation=act))
model.add(Dense(10, activation=act))
model.add(Dense(1, activation='linear'))
model.compile(optimizer=opt,loss='mean_squared_error')
    #model.fit(trainX, trainY, epochs=1, batch_size=10,shuffle=True)
    
start = time.time()
model.fit(trainX, trainY, epochs=1, shuffle=True,batch_size=batch_size, validation_data=(testX, testY), 
                callbacks=[EarlyStopping(monitor='val_loss', patience=20)], verbose=1)
    
y_pred = model.predict(testX)
end = time.time()
    
    # Calculate the RMSE score as fitness score for GA
rmse = np.sqrt(mean_squared_error(testY, y_pred))
print('Validation RMSE: ', rmse,'\n')
print ("Model took %0.2f seconds to train"%(end - start))
    
#return rmse,

# CHECK PSO

In [ ]:
import pyswarms as ps

In [ ]:
from pyswarms.utils.functions import single_obj as fx

In [ ]:
# Set-up hyperparameters
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

In [ ]:
# Call instance of PSO
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options)

In [ ]:
best_cost, best_pos = optimizer.optimize(fx.sphere, iters=100)

In [ ]:
# Obtain the cost history
optimizer.cost_history
# Obtain the position history
optimizer.pos_history
# Obtain the velocity history
optimizer.velocity_history

In [ ]:
from pyswarms.utils.plotters import plot_cost_history, plot_contour, plot_surface
plot_cost_history(optimizer.cost_history)
plt.show()